# EDA : Brazilian E-Commerce Public Dataset

In [12]:
import mysql.connector
import pandas as pd
import plotly_express as px

conn = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "cx@1076044150",
    database = "olist"
)
cursor = conn.cursor(buffered=True)

In [138]:
# 데이터 불러오기
sql = 'select * from order_info'
cursor.execute(sql)
result = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(data=result, columns=columns)
df = df.rename(columns = {'product_category_name_english' : 'category'})

In [189]:
# 카테고리별 주문 건수
df_1 = pd.pivot_table(data=df, index='category', values='customer_id', aggfunc='count')
df_1 = df_1.rename(columns = {'customer_id' : 'count'})
df_1 = df_1.sort_values(by='count', ascending=False)

px.bar(df_1, x=df_1.index, y='count',
       labels={"category":"카테고리","count":"총 주문건수"}, 
       title='카테고리별 주문 건수')

- 카테고리별 주문 건수는 bed_bath_table이 많았고, 그 다음으로는 health_beauty, sports_leisure가 많았다.
- 주문 건수가 많다는 것은 그 만큼 수요가 많다고 해석할 수 있으므로, 해당 카테고리에 대한 관심이 높은 고객은 구매 확률이 상대적으로 높을 것이다.

In [190]:
# 카테고리별 매출액 비교
df_2 = pd.pivot_table(data=df, index='category', values='price', aggfunc='sum')
df_2 = df_2.sort_values(by='price', ascending=False)

px.bar(df_2, x=df_2.index, y='price',
       labels={"category":"카테고리","price":"총 매출액"}, 
       title='카테고리별 매출액')

- 매출액은 health_beauty가 가장 높았고, watches_gifts 그 다음으로는 bed_bath_table 이였다.

In [191]:
# 카테고리별 주문 건당 매출액
df_3 = pd.merge(df_1, df_2, on='category', how='inner')
df_3['price per count'] = df_3['price'] / df_3['count']
df_3.sort_values(by='price per count', ascending=False, inplace=True)

px.bar(df_3, x=df_3.index, y='price per count',
       labels={"category":"카테고리","price":"총 매출액"}, 
       title='카테고리별 주문 건당 매출액')

- 주문 건당 매출액이 가장 높은 카테고리는 컴퓨터, 가전제품 등 전자제품이 높게 나타났다.
- 종합하면 olist의 고객은 health_beauty의 주문건수와 매출액이 높았다.

In [125]:
# 주별 주문 건수
df_4 = pd.pivot_table(data=df, index='customer_id', values='order_id', aggfunc='count')
df_4 = pd.merge(df_4, df[['customer_id', 'customer_state']], on='customer_id', how='left').drop_duplicates()
df_4 = df_4.rename(columns={'order_id':'num of orders'})

df_4_1 = pd.pivot_table(data=df_4, index='num of orders', columns='customer_state', values='customer_id', aggfunc='count')
df_4_1.sort_index(ascending=False, inplace=True)
df_4_1 = df_4_1.drop(1)

px.imshow(df_4_1, y=df_4_1.index.unique(),
       labels={"num of orders":"고객당 주문수","customer_state":"주"}, 
       title='고객당 주문건수 주별 분포', color_continuous_scale='Viridis_r')

In [179]:
# 고객당 주문건수가 2개 이상인 고객의 주문 카테고리 성향 파악
df_5 = df_4[df_4['num of orders'] > 1]['customer_id']
df_5 = df[df['customer_id'].isin(df_5)]

df_5_1 = pd.pivot_table(data=df_5, index='category', values='order_id', aggfunc='count')
df_5_1 = df_5_1.sort_values(by='order_id', ascending=False)
df_5_1 = df_5_1.rename(columns={'order_id':'count'})

px.bar(df_5_1, x=df_5_1.index, y='count',
       labels={"category":"카테고리","count":"주문건수"}, 
       title='고객 주문건수가 2회이상인 카테고리빈도')

In [188]:
# 주별 주문건당 소비금액량
df_6 = pd.pivot_table(data=df, index='customer_state', values=['order_id', 'price'], aggfunc={'order_id':'count', 'price':'sum'})
df_6['price per order'] = round(df_6['price'] / df_6['order_id'], 2)

df_6 = pd.merge(df_6, df_4_2, on='customer_state', how='left')
df_6.rename(columns={'order_id':'orders', 'customer_id':'2nd_orders'}, inplace=True)
df_6['2nd_rate'] = round(df_6['2nd_orders'] / df_6['orders'], 2)

df_6.sort_values(by='price per order', ascending=False, inplace=True)
df_6

,orders,price,price per order,2nd_orders,2nd_rate
customer_state,,,,,
PB,600,114874.10,191.46,45,0.08
AL,444,80314.81,180.89,24,0.05
AC,92,15982.95,173.73,5,0.05
RO,277,46031.64,166.18,22,0.08
PA,1079,178821.12,165.73,75,0.07
AP,82,13474.30,164.32,10,0.12
PI,539,86660.09,160.78,34,0.06
TO,314,49407.99,157.35,27,0.09
RN,529,83034.98,156.97,38,0.07


- 건당 평균 매출금액을 통해 주별로 인당 어느정도 매출이 발생하는지 파악 가능